In [149]:
%%capture
!pip install yfinance
!pip install "redshift-connector[full]" sqlalchemy-redshift
!pip install python-dotenv
!pip install psycopg2-binary

In [150]:
import os
from configparser import ConfigParser
import sqlalchemy as sa
from sqlalchemy.engine.url import URL
from sqlalchemy.exc import SQLAlchemyError, ResourceClosedError
import pandas as pd
from pathlib import Path
import yfinance as yf
from dotenv import load_dotenv
load_dotenv()

True

In [139]:
# This file is used by dotenv to load the db credentials
env_file = Path('.').resolve() / '.env'

In [140]:
# DB configuration environment variables
user = os.getenv('USERNAME')
passw = os.getenv('PASSW')
host = os.getenv('HOST')
port = os.getenv('PORT')
dbname = os.getenv('DB_NAME')
schema = "norbermv_dev_coderhouse"

### Defining some useful functions for handling the Redshift db connection.

In [133]:
def build_conn_string(
    user: str, 
    passw: str, 
    host: str, 
    port: str, 
    dbname: str
) -> URL:
    conn_string = URL.create(
        drivername="postgresql",
        username=user,
        password=passw,
        host=host,
        port=port,
        database=dbname
    )
    return conn_string

def conn_to_db(conn_str: URL) -> tuple :
    # Create an engine with the connection string
    engine = sa.create_engine(conn_str)
    try:
        # Connect to the database
        conn = engine.connect()
        return conn, engine
    except SQLAlchemyError as e:
        print(f"Error connecting to the database: {e}")
        return None, None

In [135]:
# Build the connection string, and connect to the DB
conn_str = build_conn_string(
    user, 
    passw, 
    host, 
    port,
    dbname
)

In [136]:
conn, engine = conn_to_db(conn_str)

## Create The `stock_hist`  Table

In [137]:
if conn is not None:
    try:
        # Use the connection to execute a DDL statement
        with conn.begin() as trans:
            conn.execute(
                f"""
                DROP TABLE IF EXISTS {schema}.stock_hist;
                CREATE TABLE {schema}.stock_hist (
                    Date TIMESTAMP,
                    open_price FLOAT,
                    High FLOAT,
                    Low FLOAT,
                    Close FLOAT,
                    Volume INT,
                    Dividends FLOAT,
                    Stock_Splits FLOAT
                );
                """
            )
            # Commit the transaction to ensure DDL statement is executed
            trans.commit()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()
else:
    print("Failed to connect to the database.")


## Retrieving the data from the `yfinance` API
Using the yfinance API to fetch historical stock data for Apple Inc. (ticker symbol: AAPL)

In [142]:
# Create a Ticker object for Apple Inc.
aapl = yf.Ticker("AAPL")

In [115]:
# Get historical market data for Apple
aapl_hist = aapl.history(period="1y")

# Show meta information about the history for Apple (requires history() to be called first)
aapl_history_metadata = aapl.history_metadata

In [143]:
# Rename the aapl_hist DataFrame columns to match the column names in the 'stock_hist' table
aapl_hist.columns = ['open_price', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock_Splits']

In [144]:
aapl_hist.head()

,open_price,High,Low,Close,Volume,Dividends,Stock_Splits
Date,,,,,,,
2023-01-13 00:00:00-05:00,131.296119,134.170055,130.928181,134.010941,57809700,0.0,0.0
2023-01-17 00:00:00-05:00,134.080541,136.526859,133.384435,135.184372,63646600,0.0,0.0
2023-01-18 00:00:00-05:00,136.059486,137.839530,134.279428,134.458435,69672800,0.0,0.0
2023-01-19 00:00:00-05:00,133.334724,135.492661,133.026450,134.518112,58280400,0.0,0.0
2023-01-20 00:00:00-05:00,134.528053,137.252828,133.473947,137.103653,80223600,0.0,0.0


In [129]:
conn, engine = conn_to_db(conn_str)

In [156]:

if conn is not None:
    try:
        with conn.begin() as trans:
            for index, row in aapl_hist.iterrows():
                sql = f"""
                INSERT INTO {schema}.stock_hist (Date, open_price, High, Low, Close, Volume, Dividends, Stock_Splits)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
                """
                conn.execute(sql, (index, row['open_price'], row['High'], row['Low'], row['Close'], row['Volume'], row['Dividends'], row['Stock_Splits']))
            # Commit the transaction
            trans.commit()
    except Exception as e:
        if isinstance(e, ResourceClosedError):
            print("Failed to connect to the database");
        else:
            print(f"An error occurred: {e}")
    finally:
        conn.close()